<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>